### Adapting ensemble code of Farshid 2023
*Requires cuda

---

In [ ]:
def train2Model(model,
                loaded_hbv,
               x,
                x2,
               y,
               c,
                c2,
               lossFun,
               *,
               nEpoch=500,
               miniBatch=[100, 30],
               saveEpoch=100,
               saveFolder=None,
               mode='seq2seq',
               bufftime=0,
               prcp_loss_factor = 15,
               smooth_loss_factor = 0,
               multiforcing=False):
    batchSize, rho = miniBatch
    # x- input; z - additional input; y - target; c - constant input
    if type(x) is tuple or type(x) is list:
        x, z = x
        x2, z2 = x2
    ngrid, nt, nx = x.shape
    ngrid, nt, nx2 = x2.shape
    if c is not None:
        nx = nx + c.shape[-1]
        nx2 = nx2 + c2.shape[-1]
    if batchSize >= ngrid:
        # batchsize larger than total grids
        batchSize = ngrid

    nIterEp = int(
        np.ceil(np.log(0.01) / np.log(1 - batchSize * rho / ngrid / (nt-bufftime))))
    if hasattr(model, 'ctRm'):
        if model.ctRm is True:
            nIterEp = int(
                np.ceil(
                    np.log(0.01) / np.log(1 - batchSize *
                                          (rho - model.ct) / ngrid / (nt-bufftime))))

    if torch.cuda.is_available():
        lossFun = lossFun.cuda()
        model = model.cuda()
        # loaded_hbv = loaded_hbv.cuda()

    # optim = torch.optim.Adadelta(list(model.parameters()) + list(loaded_hbv.parameters()))
    optim = torch.optim.Adadelta(list(model.parameters()))
    # loaded_hbv.zero_grad()
    model.zero_grad()
    if saveFolder is not None:
        runFile = os.path.join(saveFolder, 'run.csv')
        rf = open(runFile, 'w+')
    for iEpoch in range(1, nEpoch + 1):
        lossEp = 0
        if multiforcing is True:
            loss_prcp_Ep = 0
            # loss_pet_Ep = 0
            # loss_temp_Ep = 0
            loss_sf_Ep = 0
            # loss_smooth_Ep = 0
        t0 = time.time()
        for iIter in range(0, nIterEp):
            # training iterations
            if type(model) in [rnn.LstmCloseModel, rnn.AnnCloseModel, rnn.CNN1dLSTMmodel, rnn.CNN1dLSTMInmodel,
                               rnn.CNN1dLCmodel, rnn.CNN1dLCInmodel, rnn.CudnnInvLstmModel,
                               rnn.MultiInv_HBVModel, rnn.MultiInv_HBVTDModel, rnn.prcp_weights]:
                iGrid, iT = randomIndex(ngrid, nt, [batchSize, rho], bufftime=bufftime)
                if type(model) in [rnn.MultiInv_HBVModel, rnn.MultiInv_HBVTDModel, rnn.prcp_weights]:
                    xTrain = selectSubset(x, iGrid, iT, rho, bufftime=bufftime)
                    xTrain_hbv = selectSubset(x2, iGrid, iT, rho, bufftime=bufftime)
                else:
                    xTrain = selectSubset(x, iGrid, iT, rho, c=c)
                    xTrain_hbv = selectSubset(x2, iGrid, iT, rho, c=c2)
                yTrain = selectSubset(y, iGrid, iT, rho)
                if type(model) in [rnn.CNN1dLCmodel, rnn.CNN1dLCInmodel]:
                    zTrain = selectSubset(z, iGrid, iT=None, rho=None, LCopt=True)
                    zTrain_hbv = selectSubset(z2, iGrid, iT=None, rho=None, LCopt=True)
                elif type(model) in [rnn.CudnnInvLstmModel]: # For smap inv LSTM, HBV Inv
                    # zTrain = selectSubset(z, iGrid, iT=None, rho=None, LCopt=False)
                    zTrain = selectSubset(z, iGrid, iT=None, rho=None, LCopt=False, c=c) # Add the attributes to inv
                    zTrain_hbv = selectSubset(z2, iGrid, iT=None, rho=None, LCopt=False, c=c2) # Add the attributes to inv
                elif type(model) in [rnn.MultiInv_HBVModel]:
                    zTrain = selectSubset(z, iGrid, iT, rho, c=c)
                    zTrain_hbv = selectSubset(z2, iGrid, iT, rho, c=c2)
                elif type(model) in [rnn.MultiInv_HBVTDModel, rnn.prcp_weights]:
                    zTrain = selectSubset(z, iGrid, iT, rho, c=c, bufftime=bufftime)
                    zTrain_hbv = selectSubset(z2, iGrid, iT, rho, c=c2, bufftime=bufftime)
                else:
                    zTrain = selectSubset(z, iGrid, iT, rho)
                    zTrain_hbv = selectSubset(z2, iGrid, iT, rho)
                # loaded_hbv.train(mode=False)
                    
                    
                xP, prcp_loss, prcp_wghts = model(xTrain, zTrain, prcp_loss_factor)
                yP = loaded_hbv(xP, zTrain_hbv, prcp_loss_factor =0, smooth_loss_factor=0, multiforcing=False)
                # yP = model(xTrain, zTrain, prcp_loss_factor, smooth_loss_factor, multiforcing)
                # yP = model(xTrain, zTrain)
            if type(model) in [cnn.LstmCnn1d]:
                iGrid, iT = randomIndex(ngrid, nt, [batchSize, rho])
                xTrain = selectSubset(x, iGrid, iT, rho, c=c)
                # xTrain = rho/time * Batchsize * Ninput_var
                xTrain = xTrain.permute(1, 2, 0)
                yTrain = selectSubset(y, iGrid, iT, rho)
                # yTrain = rho/time * Batchsize * Ntraget_var
                yTrain = yTrain.permute(1, 2, 0)[:, :, int(rho/2):]
                yP = model(xTrain)
            # if type(model) in [hydroDL.model.rnn.LstmCnnCond]:
            #     iGrid, iT = randomIndex(ngrid, nt, [batchSize, rho])
            #     xTrain = selectSubset(x, iGrid, iT, rho)
            #     yTrain = selectSubset(y, iGrid, iT, rho)
            #     zTrain = selectSubset(z, iGrid, None, None)
            #     yP = model(xTrain, zTrain)
            # if type(model) in [hydroDL.model.rnn.LstmCnnForcast]:
            #     iGrid, iT = randomIndex(ngrid, nt, [batchSize, rho])
            #     xTrain = selectSubset(x, iGrid, iT, rho)
            #     yTrain = selectSubset(y, iGrid, iT + model.ct, rho - model.ct)
            #     zTrain = selectSubset(z, iGrid, iT, rho)
            #     yP = model(xTrain, zTrain)
            else:
                Exception('unknown model')
            # # consider the buff time for initialization
            # if bufftime > 0:
            #     yP = yP[bufftime:,:,:]
            ## temporary test for NSE loss
            if type(lossFun) in [crit.NSELossBatch, crit.NSESqrtLossBatch]:
                # if multiforcing==True:
                loss_sf = lossFun(yP, yTrain, iGrid)
                loss =  loss_sf + prcp_loss
                # else:
                #     loss = lossFun(yP, yTrain, iGrid)
            else:
                # if multiforcing==True:
                loss_sf = lossFun(yP, yTrain)
                loss = loss_sf + prcp_loss
                # else:
                #     loss = lossFun(yP, yTrain)
            loss.backward()
            optim.step()
            # model.zero_grad()
            optim.zero_grad()
            lossEp = lossEp + loss.item()
            try:
                loss_prcp_Ep = loss_prcp_Ep + prcp_loss.item()
            except:
                pass
            # loss_pet_Ep = loss_pet_Ep + pet_loss.item()
            # loss_temp_Ep = loss_temp_Ep + temp_loss.item()
            # loss_temp_Ep = 0
            # loss_pet_Ep = 0
            loss_sf_Ep = loss_sf_Ep + loss_sf.item()
                # loss_smooth_Ep = loss_smooth_Ep + smooth_loss.item()
            # print(iIter, '  ', loss.item())
            # if iIter == 223:
            #     print('This is the error point')
            #     print('Debug start')

            if iIter % 100 == 0:
                print('Iter {} of {}: Loss {:.3f}'.format(iIter, nIterEp, loss.item()))
        # print loss
        lossEp = lossEp / nIterEp
        # if multiforcing==True:
        loss_sf_Ep = loss_sf_Ep / nIterEp
        loss_prcp_Ep = loss_prcp_Ep / nIterEp
        # loss_pet_Ep = loss_pet_Ep / nIterEp
        # loss_temp_Ep = loss_temp_Ep / nIterEp
        logStr = 'Epoch {} Loss {:.3f}, Streamflow Loss {:.3f}, Precipitation Loss {:.3f}, time {:.2f}'.format(
            iEpoch, lossEp, loss_sf_Ep, loss_prcp_Ep,
            time.time() - t0)
        # else:
        #     logStr = 'Epoch {} Loss {:.3f}, time {:.2f}'.format(
        #         iEpoch, lossEp, time.time() - t0)
        print(logStr)
        # save model and loss
        if saveFolder is not None:
            rf.write(logStr + '\n')
            if iEpoch % saveEpoch == 0:
                # save model
                modelFile = os.path.join(saveFolder,
                                         'model_Ep' + str(iEpoch) + '.pt')
                torch.save(model, modelFile)
                # modelFile_hbv = os.path.join(saveFolder,
                #                          'model_hbv_Ep' + str(iEpoch) + '.pt')
                # torch.save(loaded_hbv, modelFile_hbv)
    if saveFolder is not None:
        rf.close()
    return model

all of the train2lstm_3.py 

In [ ]:
class prcp_weights(torch.nn.Module):
    def __init__(self, *, ninv, hiddeninv, drinv=0.5, prcp_datatypes=1):
        super(prcp_weights, self).__init__()
        self.ninv = ninv
        self.prcp_datatypes = prcp_datatypes

        ntp = prcp_datatypes*3
        self.hiddeninv = hiddeninv

        self.lstminv = CudnnLstmModel(
            nx=ninv, ny=ntp, hiddenSize=hiddeninv, dr=drinv)
        lb_prcp = [0.95]
        ub_prcp = [1.05]
        self.RangeBoundLoss = RangeBoundLoss(lb=lb_prcp, ub=ub_prcp)

    def forward(self, x, z, prcp_loss_factor):
        z.requires_grad = True

        wghts = self.lstminv(z)
        ntstep = wghts.shape[0]
        ngage = wghts.shape[1]
        wghts_scaled = torch.sigmoid(wghts)
        prcp_wavg = torch.zeros((ntstep, ngage), requires_grad=True, dtype=torch.float32).cuda()
        # prcp_wghts_sum = torch.sum(wghts_scaled, dim=2)


        temp_wavg = torch.zeros((ntstep, ngage), requires_grad=True, dtype=torch.float32).cuda()
        pet_wavg = torch.zeros((ntstep, ngage), requires_grad=True, dtype=torch.float32).cuda()
        prcp_wghts_sum = torch.sum(wghts_scaled[:,:,:3], dim=2)
        temp_wghts_sum = torch.sum(wghts_scaled[:,:,3:6], dim=2)
        pet_wghts_sum = torch.sum(wghts_scaled[:,:,6:], dim=2)



        for para in range(wghts.shape[2]):
            if para<3:
                prcp_wavg = prcp_wavg + wghts_scaled[:, :, para] * x[:, :, para]
            elif para<6:
                temp_wavg = temp_wavg + wghts_scaled[:, :, para] * x[:, :, para]
            elif para>=6:
                pet_wavg = pet_wavg + wghts_scaled[:, :, para] * x[:, :, para]

        x_new = torch.empty((ntstep, ngage, 3), requires_grad=True, dtype=torch.float32).cuda()
        x_new[:, :, 0] = prcp_wavg
        # x_new[:, :, 1] = x[:, :, self.prcp_datatypes]
        # x_new[:, :, 2] = x[:, :, -1]
        x_new[:, :, 1] = temp_wavg
        x_new[:, :, 2] = pet_wavg
        # range_bound_loss_prcp = self.RangeBoundLoss([prcp_wghts_sum], factor=prcp_loss_factor)+self.RangeBoundLoss([temp_wghts_sum], factor=prcp_loss_factor)+self.RangeBoundLoss([pet_wghts_sum], factor=prcp_loss_factor)
        range_bound_loss_prcp = self.RangeBoundLoss([prcp_wghts_sum], factor=prcp_loss_factor)

        # range_bound_loss_prcp = 0

        grad_daymet = autograd.grad(outputs=wghts_scaled[:, :, 0], inputs=z, grad_outputs=torch.ones_like(wghts_scaled[:, :, 0]), retain_graph=True)[0]
        grad_maurer = autograd.grad(outputs=wghts_scaled[:, :, 1], inputs=z, grad_outputs=torch.ones_like(wghts_scaled[:, :, 1]), retain_graph=True)[0]
        grad_nldas = autograd.grad(outputs=wghts_scaled[:, :, 2], inputs=z, grad_outputs=torch.ones_like(wghts_scaled[:, :, 2]), retain_graph=True)[0]

        return x_new, range_bound_loss_prcp, wghts_scaled, grad_daymet, grad_maurer, grad_nldas
        # return x_new, range_bound_loss_prcp, wghts_scaled


In [ ]:
prcp_weights